In [2]:
import pandas as pd
import numpy as np
import bs4
import requests
from shapely.geometry import Point
import geopandas as gpd
import warnings

In [3]:
response_df1 = requests.get("https://datos.madrid.es/egob/catalogo/202318-0-escuelas-infantiles.json").json()
response_df1.keys()

dict_keys(['@context', '@graph'])

In [4]:
res_df1 = pd.json_normalize(response_df1["@graph"])

In [5]:
response_df2 = requests.get("https://datos.madrid.es/egob/catalogo/202311-0-colegios-publicos.json").json()
response_df2.keys()

dict_keys(['@context', '@graph'])

In [6]:
res_df2 = pd.json_normalize(response_df2["@graph"])

In [7]:
escuelas_colegios_df = pd.concat([res_df1, res_df2])

In [8]:
new_places_df = escuelas_colegios_df.loc[:,['id','title','address.locality','address.postal-code','address.street-address','location.latitude','location.longitude']]
new_places_df['key'] = int(1)
display (new_places_df)

,id,title,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,key
0,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1
1,4549323,Escuela infantil municipal Altair,MADRID,28044,CALLE JOSE DE CADALSO 43,40.378132,-3.768701,1
2,5299172,Escuela infantil municipal Amanecer,MADRID,28032,CALLE TITANIO 7,40.401990,-3.601044,1
3,5633312,Escuela infantil municipal Ana de Austria,MADRID,28050,CALLE ANA DE AUSTRIA 32,40.495522,-3.655737,1
4,4724002,Escuela infantil municipal Campanilla,MADRID,28020,CALLE AVILA 4,40.452700,-3.701998,1
...,...,...,...,...,...,...,...,...
243,5326,Colegio Público Virgen del Cortijo,MADRID,28050,AVENIDA MANOTERAS 63,40.487254,-3.660022,1
244,5319,Colegio Público Vázquez de Mella,MADRID,28005,CALLE BAILEN 18,40.412095,-3.713999,1
245,4690940,Colegio Público Winston Churchill,MADRID,28032,AVENIDA GRAN VIA DEL ESTE 5,40.402966,-3.595314,1
246,5096,Colegio Público de Practicas Asunción Rincón,MADRID,28003,AVENIDA FILIPINAS 5,40.439916,-3.710968,1


In [8]:
new_places_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 0 to 247
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      309 non-null    object 
 1   title                   309 non-null    object 
 2   address.locality        309 non-null    object 
 3   address.postal-code     309 non-null    object 
 4   address.street-address  309 non-null    object 
 5   location.latitude       309 non-null    float64
 6   location.longitude      309 non-null    float64
 7   key                     309 non-null    int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 21.7+ KB


In [12]:
bicimad_original_df = pd.read_csv("../data/dbo.bicimad_stations.csv")
bicimad_df = bicimad_original_df.loc[:,['id','name','address','dock_bikes','free_bases','geometry_coordinates']]

In [13]:
bicimad_df[['location.longitude','location.latitude']] = bicimad_df['geometry_coordinates'].str.split(',',expand=True)
bicimad_df['location.longitude'] = bicimad_df['location.longitude'].str.replace("[","")
bicimad_df['location.latitude'] = bicimad_df['location.latitude'].str.replace("]","")
bicimad_df = bicimad_df.astype({'location.latitude': float, 'location.longitude': float})
new_bicimad_df = bicimad_df.loc[:,['id','name','address','dock_bikes','free_bases','location.longitude','location.latitude']]
new_bicimad_df['key'] = int(1)
display(new_bicimad_df)
warnings.filterwarnings('ignore')

,id,name,address,dock_bikes,free_bases,location.longitude,location.latitude,key
0,1,Puerta del Sol A,Puerta del Sol nº 1,15,11,-3.701834,40.417214,1
1,2,Puerta del Sol B,Puerta del Sol nº 1,22,4,-3.701603,40.417313,1
2,3,Miguel Moya,Calle Miguel Moya nº 1,12,11,-3.705842,40.420589,1
3,4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,4,11,-3.706917,40.430294,1
4,5,Malasaña,Calle Manuela Malasaña nº 5,12,12,-3.702587,40.428552,1
...,...,...,...,...,...,...,...,...
259,266,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,0,24,-3.726990,40.443750,1
260,267,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,12,12,-3.726930,40.443420,1
261,268,Facultad Biología,Calle José Antonio Novais frente al nº 12,9,14,-3.727295,40.448332,1
262,269,Facultad Derecho,Avenida Complutense nº 23,5,18,-3.729370,40.451090,1


In [15]:
new_bicimad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  264 non-null    int64  
 1   name                264 non-null    object 
 2   address             264 non-null    object 
 3   dock_bikes          264 non-null    int64  
 4   free_bases          264 non-null    int64  
 5   location.longitude  264 non-null    float64
 6   location.latitude   264 non-null    float64
 7   key                 264 non-null    int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 16.6+ KB


In [16]:
%%time
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

new_places_df['mercator'] = new_places_df.apply(lambda new_places_df: to_mercator(new_places_df['location.latitude'],new_places_df['location.longitude']), axis = 1)
display(new_places_df)

,id,title,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,key,mercator
0,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283)
1,4549323,Escuela infantil municipal Altair,MADRID,28044,CALLE JOSE DE CADALSO 43,40.378132,-3.768701,1,POINT (4494873.081456062 -419832.6705785314)
2,5299172,Escuela infantil municipal Amanecer,MADRID,28032,CALLE TITANIO 7,40.401990,-3.601044,1,POINT (4497528.948937292 -401130.596425248)
3,5633312,Escuela infantil municipal Ana de Austria,MADRID,28050,CALLE ANA DE AUSTRIA 32,40.495522,-3.655737,1,POINT (4507940.837033544 -407231.1670990717)
4,4724002,Escuela infantil municipal Campanilla,MADRID,28020,CALLE AVILA 4,40.452700,-3.701998,1,POINT (4503174.019987035 -412391.5771022496)
...,...,...,...,...,...,...,...,...,...
243,5326,Colegio Público Virgen del Cortijo,MADRID,28050,AVENIDA MANOTERAS 63,40.487254,-3.660022,1,POINT (4507020.494757262 -407709.1747154693)
244,5319,Colegio Público Vázquez de Mella,MADRID,28005,CALLE BAILEN 18,40.412095,-3.713999,1,POINT (4498653.885099497 -413730.2642768721)
245,4690940,Colegio Público Winston Churchill,MADRID,28032,AVENIDA GRAN VIA DEL ESTE 5,40.402966,-3.595314,1,POINT (4497637.552307622 -400491.4291664587)
246,5096,Colegio Público de Practicas Asunción Rincón,MADRID,28003,AVENIDA FILIPINAS 5,40.439916,-3.710968,1,POINT (4501750.852518209 -413392.1668429679)


In [17]:
%%time
new_bicimad_df['mercator'] = new_bicimad_df.apply(lambda new_bicimad_df: to_mercator(new_bicimad_df['location.latitude'],new_bicimad_df['location.longitude']), axis = 1)
display(new_bicimad_df)

,id,name,address,dock_bikes,free_bases,location.longitude,location.latitude,key,mercator
0,1,Puerta del Sol A,Puerta del Sol nº 1,15,11,-3.701834,40.417214,1,POINT (4499223.648366921 -412373.2851572837)
1,2,Puerta del Sol B,Puerta del Sol nº 1,22,4,-3.701603,40.417313,1,POINT (4499234.670122574 -412347.498528782)
2,3,Miguel Moya,Calle Miguel Moya nº 1,12,11,-3.705842,40.420589,1,POINT (4499599.340516399 -412820.3206106317)
3,4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,4,11,-3.706917,40.430294,1,POINT (4500679.707306497 -412940.3068148255)
4,5,Malasaña,Calle Manuela Malasaña nº 5,12,12,-3.702587,40.428552,1,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...,...,...,...,...,...
259,266,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,0,24,-3.726990,40.443750,1,POINT (4502177.655770458 -415179.5220676512)
260,267,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,12,12,-3.726930,40.443420,1,POINT (4502140.920338497 -415172.8287427764)
261,268,Facultad Biología,Calle José Antonio Novais frente al nº 12,9,14,-3.727295,40.448332,1,POINT (4502687.743941171 -415213.4906984274)
262,269,Facultad Derecho,Avenida Complutense nº 23,5,18,-3.729370,40.451090,1,POINT (4502994.74083288 -415445.0243226733)


In [18]:
%%time
total_df = pd.merge(new_places_df,new_bicimad_df, on="key", how="outer")
display(total_df)

,id_x,title,address.locality,address.postal-code,address.street-address,location.latitude_x,location.longitude_x,key,mercator_x,id_y,name,address,dock_bikes,free_bases,location.longitude_y,location.latitude_y,mercator_y
0,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),1,Puerta del Sol A,Puerta del Sol nº 1,15,11,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837)
1,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),2,Puerta del Sol B,Puerta del Sol nº 1,22,4,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782)
2,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),3,Miguel Moya,Calle Miguel Moya nº 1,12,11,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317)
3,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,4,11,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148255)
4,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),5,Malasaña,Calle Manuela Malasaña nº 5,12,12,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81571,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),266,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,0,24,-3.726990,40.443750,POINT (4502177.655770458 -415179.5220676512)
81572,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),267,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,12,12,-3.726930,40.443420,POINT (4502140.920338497 -415172.8287427764)
81573,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),268,Facultad Biología,Calle José Antonio Novais frente al nº 12,9,14,-3.727295,40.448332,POINT (4502687.743941171 -415213.4906984274)
81574,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),269,Facultad Derecho,Avenida Complutense nº 23,5,18,-3.729370,40.451090,POINT (4502994.74083288 -415445.0243226733)


In [23]:
%%time
def distance_meters(start,finish):
    # return the distance in metres between to two points
    return start.distance(finish)

total_df['distance'] = total_df.apply(lambda total_df: distance_meters(total_df['mercator_x'],total_df['mercator_y']), axis = 1)
display(total_df)

,id_x,title,address.locality,address.postal-code,address.street-address,location.latitude_x,location.longitude_x,key,mercator_x,id_y,name,address,dock_bikes,free_bases,location.longitude_y,location.latitude_y,mercator_y,distance
0,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),1,Puerta del Sol A,Puerta del Sol nº 1,15,11,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837),4187.097788
1,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),2,Puerta del Sol B,Puerta del Sol nº 1,22,4,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782),4181.568956
2,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),3,Miguel Moya,Calle Miguel Moya nº 1,12,11,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317),3746.865547
3,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,4,11,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148255),2659.877980
4,58241,Escuela infantil municipal Agua Dulce,MADRID,28039,CALLE LEÑEROS 25,40.454061,-3.709371,1,POINT (4503325.457708918 -413214.0871607283),5,Malasaña,Calle Manuela Malasaña nº 5,12,12,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012),2938.700507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81571,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),266,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,0,24,-3.726990,40.443750,POINT (4502177.655770458 -415179.5220676512),13584.465918
81572,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),267,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,12,12,-3.726930,40.443420,POINT (4502140.920338497 -415172.8287427764),13571.244295
81573,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),268,Facultad Biología,Calle José Antonio Novais frente al nº 12,9,14,-3.727295,40.448332,POINT (4502687.743941171 -415213.4906984274),13719.637536
81574,4690849,Colegio Público Álvaro de Bazan,MADRID,28022,CALLE ZURICH 10,40.421543,-3.607242,1,POINT (4499705.618399205 -401821.8746131732),269,Facultad Derecho,Avenida Complutense nº 23,5,18,-3.729370,40.451090,POINT (4502994.74083288 -415445.0243226733),14014.582919


Wall time: 2.52 s


In [20]:
total_sorted_df = total_df.sort_values(["title", "distance"], ascending = (True, True))
reindex_total_sorted_df = total_sorted_df.reset_index(drop=True)
display(reindex_total_sorted_df)

,id_x,title,address.locality,address.postal-code,address.street-address,location.latitude_x,location.longitude_x,key,mercator_x,id_y,name,address,dock_bikes,free_bases,location.longitude_y,location.latitude_y,mercator_y,distance
0,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),257,Manuel Caldeiro,Paseo de la Castellana nº 298,16,7,-3.685320,40.479380,POINT (4506143.969227422 -410531.1174823334),3796.394240
1,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),260,Chamartín,Calle Rodríguez Jaén nº 40,7,15,-3.683910,40.471600,POINT (4505277.903589051 -410373.8318768776),3976.932693
2,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),225,Pedro Rico,Calle Pedro Rico nº 4,13,10,-3.688463,40.481086,POINT (4506333.880278716 -410881.7202647889),4092.005819
3,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),262,Pío XII,Avenida Pío XII nº 3,11,12,-3.677120,40.460410,POINT (4504032.238487074 -409616.4103570179),4166.338771
4,176850,Colegio Público Adolfo Suárez,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),256,Torre Cepsa,Calle Sinesio Delgado nº 2,14,9,-3.687970,40.474350,POINT (4505584.032188732 -410826.7258551974),4257.242824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81571,4552561,Escuela infantil municipal VeoVeo,MADRID,28027,CALLE CONDESA DE VENADITO 3,40.444110,-3.655331,1,POINT (4502217.688052606 -407185.9294373856),265,INEF,Avenida Juan de Herrera frente a la calle Paul...,14,9,-3.729970,40.438960,POINT (4501644.435409558 -415511.9577777162),8345.739423
81572,4552561,Escuela infantil municipal VeoVeo,MADRID,28027,CALLE CONDESA DE VENADITO 3,40.444110,-3.655331,1,POINT (4502217.688052606 -407185.9294373856),220,Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9,12,10,-3.716591,40.398247,POINT (4497112.284980891 -414019.4641230708),8530.084221
81573,4552561,Escuela infantil municipal VeoVeo,MADRID,28027,CALLE CONDESA DE VENADITO 3,40.444110,-3.655331,1,POINT (4502217.688052606 -407185.9294373856),231,Ermita Santo,Paseo de la Ermita del Santo nº 48,13,8,-3.724489,40.406838,POINT (4498068.615450309 -414900.5668678451),8759.590980
81574,4552561,Escuela infantil municipal VeoVeo,MADRID,28027,CALLE CONDESA DE VENADITO 3,40.444110,-3.655331,1,POINT (4502217.688052606 -407185.9294373856),224,Puerta del Ángel,Plaza Puerta del Ángel nº 4,16,7,-3.728318,40.413764,POINT (4498839.629519532 -415327.6677751078),8814.713984


In [21]:
groupby_df = total_df.groupby(["title"])["distance"].min()
display(groupby_df)
len(groupby_df)

title
Colegio Público Adolfo Suárez                      3796.394240
Colegio Público Agustina Díez                      1932.123857
Colegio Público Agustín Rodriguez Sahagún          3856.153931
Colegio Público Alberto Alcocer                    3253.471117
Colegio Público Alcalde de Móstoles                6261.761115
                                                      ...     
Escuela infantil municipal Osa Menor                182.720201
Escuela infantil municipal Parque de Ingenieros    3798.993043
Escuela infantil municipal Ruiz Jiménez             188.194006
Escuela infantil municipal Tarabilla                949.064501
Escuela infantil municipal VeoVeo                   255.295615
Name: distance, Length: 309, dtype: float64

309

In [26]:
total_groupby_df = pd.DataFrame(groupby_df)
total_groupby_df_windex = total_groupby_df.reset_index(drop = False)
display(total_groupby_df_windex)

,title,distance
0,Colegio Público Adolfo Suárez,3796.394240
1,Colegio Público Agustina Díez,1932.123857
2,Colegio Público Agustín Rodriguez Sahagún,3856.153931
3,Colegio Público Alberto Alcocer,3253.471117
4,Colegio Público Alcalde de Móstoles,6261.761115
...,...,...
304,Escuela infantil municipal Osa Menor,182.720201
305,Escuela infantil municipal Parque de Ingenieros,3798.993043
306,Escuela infantil municipal Ruiz Jiménez,188.194006
307,Escuela infantil municipal Tarabilla,949.064501


In [39]:
final_df = total_groupby_df_windex.merge(reindex_total_sorted_df,how="left")
display(final_df)

,title,distance,id_x,address.locality,address.postal-code,address.street-address,location.latitude_x,location.longitude_x,key,mercator_x,id_y,name,address,dock_bikes,free_bases,location.longitude_y,location.latitude_y,mercator_y
0,Colegio Público Adolfo Suárez,3796.394240,176850,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,1,POINT (4507170.6317725 -406876.1792826843),257,Manuel Caldeiro,Paseo de la Castellana nº 298,16,7,-3.685320,40.479380,POINT (4506143.969227422 -410531.1174823334)
1,Colegio Público Agustina Díez,1932.123857,5075,MADRID,28038,CALLE VESUBIO 5,40.388191,-3.647237,1,POINT (4495992.849193973 -406283.0125532473),234,Sierra Toledana,Camino de Valderribas nº 94,10,14,-3.660777,40.399015,POINT (4497197.81342333 -407793.364764792)
2,Colegio Público Agustín Rodriguez Sahagún,3856.153931,4693138,MADRID,28031,CALLE LA GRANJA DE SAN ILDEFONSO 17,40.368040,-3.622168,1,POINT (4493749.655446447 -403486.7215186193),229,Pavones,Calle Hacienda de Pavones nº 342,8,14,-3.634587,40.400368,POINT (4497348.393620864 -404872.003688147)
3,Colegio Público Alberto Alcocer,3253.471117,5076,MADRID,28022,AVENIDA CANILLEJAS A VICALVARO 56,40.433338,-3.612052,1,POINT (4501018.577505093 -402358.4139760713),223,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,18,5,-3.640994,40.429706,POINT (4500614.284841758 -405586.6676448087)
4,Colegio Público Alcalde de Móstoles,6261.761115,5077,MADRID,28044,CALLE ALDEANUEVA DE LA VERA 9,40.379837,-3.776448,1,POINT (4495062.882345807 -420696.9779405064),232,Caramuel,Calle Caramuel nº 51,14,9,-3.728534,40.409136,POINT (4498324.476616229 -415351.7319152079)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Escuela infantil municipal Osa Menor,182.720201,187607,MADRID,28004,CALLE MEJIA LEQUERICA 21,40.428040,-3.699502,1,POINT (4500428.821429083 -412113.1013101297),58,Barceló,Calle Barceló nº 7,18,2,-3.700423,40.426683,POINT (4500277.743757191 -412215.8739185985)
305,Escuela infantil municipal Parque de Ingenieros,3798.993043,11508438,MADRID,28041,CALLE LA TABERNERA DEL PUERTO 26,40.355056,-3.704144,1,POINT (4492304.303298196 -412630.916716107),221,Glorieta de Cádiz,Calle Antonio López nº 129,7,16,-3.700170,40.388950,POINT (4496077.347674986 -412187.6512511889)
306,Escuela infantil municipal Ruiz Jiménez,188.194006,88758,MADRID,28028,CALLE JOSE PICON 17,40.435093,-3.672375,1,POINT (4501213.942059446 -409087.0970815064),199,Diego de león,Calle Alonso Heredia nº 1,5,11,-3.674000,40.434639,POINT (4501163.401625942 -409268.3776613389)
307,Escuela infantil municipal Tarabilla,949.064501,168220,MADRID,28017,CALLE GANDHI 14,40.423692,-3.634964,1,POINT (4499944.775217957 -404914.0010923386),223,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,18,5,-3.640994,40.429706,POINT (4500614.284841758 -405586.6676448087)


In [48]:
project1_fdf = pd.DataFrame(columns = ["Place of interest","Place address","BiciMAD station","Station location","Distance"])
project1_fdf[["Place of interest","Place address","BiciMAD station","Station location","Distance"]] = final_df[["title","address.street-address","name","address","distance"]]
display(project1_fdf)

,Place of interest,Place address,BiciMAD station,Station location,Distance
0,Colegio Público Adolfo Suárez,CALLE INFANTA CATALINA MICAELA 31,Manuel Caldeiro,Paseo de la Castellana nº 298,3796.394240
1,Colegio Público Agustina Díez,CALLE VESUBIO 5,Sierra Toledana,Camino de Valderribas nº 94,1932.123857
2,Colegio Público Agustín Rodriguez Sahagún,CALLE LA GRANJA DE SAN ILDEFONSO 17,Pavones,Calle Hacienda de Pavones nº 342,3856.153931
3,Colegio Público Alberto Alcocer,AVENIDA CANILLEJAS A VICALVARO 56,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,3253.471117
4,Colegio Público Alcalde de Móstoles,CALLE ALDEANUEVA DE LA VERA 9,Caramuel,Calle Caramuel nº 51,6261.761115
...,...,...,...,...,...
304,Escuela infantil municipal Osa Menor,CALLE MEJIA LEQUERICA 21,Barceló,Calle Barceló nº 7,182.720201
305,Escuela infantil municipal Parque de Ingenieros,CALLE LA TABERNERA DEL PUERTO 26,Glorieta de Cádiz,Calle Antonio López nº 129,3798.993043
306,Escuela infantil municipal Ruiz Jiménez,CALLE JOSE PICON 17,Diego de león,Calle Alonso Heredia nº 1,188.194006
307,Escuela infantil municipal Tarabilla,CALLE GANDHI 14,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,949.064501
